In [101]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

import xgboost

import random
import os

warnings.filterwarnings(action='ignore')

In [102]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [103]:
path= '/opt/ml/level1_bookratingprediction_recsys-level1-recsys-06/data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
train_ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('train_ratings shape: ', train_ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
train_ratings shape:  (306795, 3)


In [104]:
def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)


In [105]:
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.drop(['location'], axis=1)

In [106]:
ratings = pd.concat([train_ratings, test_ratings]).reset_index(drop=True)
context_df = ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
train_df = train_ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
test_df = test_ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')

In [107]:
loc_city2idx = {v:k for k,v in enumerate(context_df['location_city'].unique())}
loc_state2idx = {v:k for k,v in enumerate(context_df['location_state'].unique())}
loc_country2idx = {v:k for k,v in enumerate(context_df['location_country'].unique())}

In [108]:
train_df['location_city'] = train_df['location_city'].map(loc_city2idx)
train_df['location_state'] = train_df['location_state'].map(loc_state2idx)
train_df['location_country'] = train_df['location_country'].map(loc_country2idx)

test_df['location_city'] = test_df['location_city'].map(loc_city2idx)
test_df['location_state'] = test_df['location_state'].map(loc_state2idx)
test_df['location_country'] = test_df['location_country'].map(loc_country2idx)

In [109]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [110]:
train_df['age'] = train_df['age'].fillna(int(train_df['age'].mean()))
train_df['age'] = train_df['age'].apply(age_map)
test_df['age'] = test_df['age'].fillna(int(test_df['age'].mean()))
test_df['age'] = test_df['age'].apply(age_map)

In [111]:
category2idx = {v:k for k,v in enumerate(context_df['category'].unique())}
publisher2idx = {v:k for k,v in enumerate(context_df['publisher'].unique())}
language2idx = {v:k for k,v in enumerate(context_df['language'].unique())}
author2idx = {v:k for k,v in enumerate(context_df['book_author'].unique())}

In [112]:
train_df['category'] = train_df['category'].map(category2idx)
train_df['publisher'] = train_df['publisher'].map(publisher2idx)
train_df['language'] = train_df['language'].map(language2idx)
train_df['book_author'] = train_df['book_author'].map(author2idx)

test_df['category'] = test_df['category'].map(category2idx)
test_df['publisher'] = test_df['publisher'].map(publisher2idx)
test_df['language'] = test_df['language'].map(language2idx)
test_df['book_author'] = test_df['book_author'].map(author2idx)

In [113]:
encode_columns = ['isbn']

tr_encode_df = train_df[encode_columns]
te_encode_df = test_df[encode_columns]

tr_encode_df = tr_encode_df.astype('str')
te_encode_df = te_encode_df.astype('str')

tr_encode_df = tr_encode_df.apply(LabelEncoder().fit_transform)
te_encode_df = te_encode_df.apply(LabelEncoder().fit_transform)

trscore_encode_drop = train_df.drop(encode_columns, axis = 1)
tsscore_encode_drop = test_df.drop(encode_columns, axis = 1)
train_df = pd.concat([trscore_encode_drop, tr_encode_df ], axis = 1)
test_df = pd.concat([tsscore_encode_drop, te_encode_df ], axis = 1)

In [114]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   user_id           306795 non-null  int64
 1   rating            306795 non-null  int64
 2   age               306795 non-null  int64
 3   location_city     306795 non-null  int64
 4   location_state    306795 non-null  int64
 5   location_country  306795 non-null  int64
 6   category          306795 non-null  int64
 7   publisher         306795 non-null  int64
 8   language          306795 non-null  int64
 9   book_author       306795 non-null  int64
 10  isbn              306795 non-null  int64
dtypes: int64(11)
memory usage: 28.1 MB


In [115]:
train_x, val_x, train_y, val_y = train_test_split(train_df.drop(['rating'], axis = 1), train_df['rating'], test_size=0.2, random_state=42)
# train_ds = lgb.Dataset(train_x, label = train_y) 
# val_ds = lgb.Dataset(val_x, label = val_y) 

In [116]:
# params = {'learning_rate': 0.01, 
#           'max_depth': 16, 
#           'boosting': 'gbdt', 
#           'objective': 'regression', 
#           'metric': 'rmse', 
#           'is_training_metric': True, 
#           'num_leaves': 144, 
#           'feature_fraction': 0.9, 
#           'bagging_fraction': 0.7, 
#           'bagging_freq': 5, 
#           'seed':42}

In [117]:
xgb_model = xgboost.XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=16)

xgb_model.fit(train_x,train_y)
# xgboost.plot_importance(xgb_model)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=16, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=2000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [118]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245436 entries, 121312 to 121958
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   user_id           245436 non-null  int64
 1   age               245436 non-null  int64
 2   location_city     245436 non-null  int64
 3   location_state    245436 non-null  int64
 4   location_country  245436 non-null  int64
 5   category          245436 non-null  int64
 6   publisher         245436 non-null  int64
 7   language          245436 non-null  int64
 8   book_author       245436 non-null  int64
 9   isbn              245436 non-null  int64
dtypes: int64(10)
memory usage: 20.6 MB


In [119]:
predictions = xgb_model.predict(val_x)
predictions

array([7.1242933, 7.40239  , 7.6620193, ..., 6.2354946, 7.549912 ,
       6.417709 ], dtype=float32)

In [120]:
r_sq = xgb_model.score(train_x, train_y)
print(mean_squared_error(predictions,val_y)**0.5)

2.1891611468893006


In [122]:
submit = pd.read_csv(path + 'sample_submission.csv')
pred = xgb_model.predict(test_df.drop(['rating'],axis = 1))
test_df['rating'] = pred
submit['rating'] = test_df['rating']
submit.to_csv('../data/20221027_Catboost_Plueid_real.csv', index = False)